McKenzie Maidl

DATA 5310

Homework 7

In [1]:
# imports
import pandas as pd
import altair as alt
import geopandas as gpd
import json
import pyproj
import osmnx as ox
import folium
from folium import plugins

# settings
pd.set_option('display.max_columns', None)

In [2]:
# base map
m = folium.Map(location=[44.97, -93.29], tiles='cartodbpositron', zoom_start=12, control_scale=True)

In [3]:
# Metropolitan Council Equity Considerations - shapefile to geojson
povshp = gpd.read_file('Data/equity_considerations_4326_mpls/equity_considerations_4326_mpls.shp')
povshp.to_file('Data/pov.geojson', driver='GeoJSON')

# create poverty dataframe
pov_data = json.load(open('Data/pov.geojson'))
pov_df = pd.DataFrame(pov_data['features'])
pov = pd.json_normalize(pov_df['properties'])[['GEOID10', 'CTU_PRMRY', 'PPOV185']]
pov_mpls = pov[pov['CTU_PRMRY'] == 'Minneapolis']

# create poverty choropleth
#      PPOV185 - People whose family income is less than 185% of the federal poverty threshold, as a 
#      proportion of all people for whom poverty status is determined.

pov_chor = folium.Choropleth(
    geo_data=pov_data,
    name='choropleth',
    data=pov_mpls,
    columns=['GEOID10','PPOV185'],
    key_on='properties.GEOID10',
    fill_color='PuBuGn',
    line_color='white',
    fill_opacity=0.5,
    line_opacity=0.8,
    legend_name='Percent of People in Census Tract with Family Income <185% Federal Poverty Threshold',
)

pov_chor.add_to(m)

In [4]:
# Minneapolis border (opendata.minneapolismn.gov)
mpls = folium.GeoJson('Data/MPLS_City_Boundary.geojson', style_function=lambda feature: {
        'fillColor': '#00000000',
        'color': '#a2abae',
        'weight': 2})

mpls.add_to(m)

In [5]:
# Minneapolis geometry
mpls_geom = ox.geocode_to_gdf('R136712', by_osmid = True)

# subset of amenities
pd.options.mode.chained_assignment = None
sub = ['drinking_water', 'toilets', 'waste_basket', 'shelter']
amenities = ox.features_from_polygon(mpls_geom['geometry'].iloc[0], tags={'amenity': sub})
amenities = gpd.GeoDataFrame.from_features(amenities)

# point data
amenitypts = amenities[amenities.geom_type == 'Point']
amenitypts['lon']=amenitypts['geometry'].x
amenitypts['lat']=amenitypts['geometry'].y

# remove non-accessible points
amenitypts = amenitypts[~amenitypts.access.isin(['no', 'customers', 'permissive', 'private'])]

# clustering data
marker_cluster = plugins.MarkerCluster().add_to(m)

# create icons for different ammenities
for idx, row in amenitypts.iterrows():
    lon = row['lon']
    lat = row['lat']
    
    if row['amenity'] == 'drinking_water':
        icon = folium.Icon(icon = 'glass-water-droplet', color = 'lightblue', prefix = 'fa')
        name = 'Drinking Water'
    elif row['amenity'] == 'toilets':
        icon = folium.Icon(icon = 'toilet', color = 'green', prefix = 'fa')
        name = 'Public Restroom'
    elif row['amenity'] == 'waste_basket':
        icon = folium.Icon(icon = 'trash', color = 'orange', prefix = 'fa')
        name = 'Waste Basket'
    elif row['amenity'] == 'shelter':
        icon = folium.Icon(icon = 'person-shelter', color = 'pink', prefix = 'fa')
        if row['shelter_type'] == 'public_transport':
            name = 'Shelter (transit)'
        elif row['shelter_type'] == 'picnic_shelter':
            name = 'Shelter (picnic)'
        else:
            name = 'Shelter (other)'
    
    folium.Marker(location=[lat, lon], tooltip=name, icon=icon).add_to(marker_cluster)

In [6]:
# box for text
folium.Rectangle([(45.042, -93.463), (44.920, -93.345)], 
                 weight=1,
                 color='white', 
                 fill=True, 
                 fill_color='white', 
                 fill_opacity=0.4).add_to(m)

In [7]:
# adding text to map
title = folium.Marker(location=[45.04, -93.46],
              icon=folium.DivIcon(html='<b>Public Amenities and Poverty in Minneapolis</b>',
                                  class_name='mapTitle'),
              )
title.add_to(m)

text = folium.Marker(location=[45.03, -93.46],
              icon=folium.DivIcon(html="""
              <p>
              This map combines the locations of public amenities with<br>
              poverty rates in Minneapolis, MN. Amenities include water,<br>
              public restrooms, waste bins, and shelters, each of which<br>
              is essential for all people, but particularly those without<br>
              access to private facilities. Poverty rates are shown at<br>
              the census tract level, and are measured by the percent<br>
              of individuals with families whose incomes are less than<br>
              185% of the federal poverty level. All census tracts that<br>
              are partially or fully within Minneapolis are included.<br>
              </p>
              <p>
              Each amenity is encoded with a point at its latitude and<br>
              longitude. In addition, a color encoding separates the<br>
              amenities by type. This is also shown with a tooltip of<br>
              the amenity type. The location encoding allows viewers<br>
              to understand where amenities are more concentrated or<br>
              lacking. The color and tooltip encodings further increase<br>
              understanding. Poverty is encoded with both location and<br>
              color. Location is aggregated into polygon features<br>
              representing census tracts, each of which has about<br>
              4,000 people. Census tracts are colored by the percent<br>
              of the population in poverty. These two encodings allow<br>
              readers to view patterns on a local level - comparing the<br>
              poverty rate and amenities in a census tract - and across<br>
              the city.
              </p>""",
                                  class_name='mapText'),
              )
text.add_to(m)

m.get_root().html.add_child(folium.Element("""
<style>
.mapTitle {
    white-space: nowrap;
    font-size:medium
}
.mapText {
    white-space: nowrap;
    font-size:small
}
</style>
"""))

In [8]:
# name and sources
name = folium.Marker(location=[44.915, -93.46],
              icon=folium.DivIcon(html='McKenzie Maidl',
                                  class_name='name'),
              )
name.add_to(m)

sources = folium.Marker(location=[44.912, -93.46],
              icon=folium.DivIcon(html='Sources: OpenStreetMap, Metropolitan Council (2017-2019)',
                                  class_name='mapSources'),
              )
sources.add_to(m)

m.get_root().html.add_child(folium.Element("""
<style>
.mapSources {
    white-space: nowrap
}
.name {
    white-space: nowrap
}
</style>
"""))

In [9]:
# show map
m

In [10]:
# save map
m.save('hw7_map.html')